# Thai Wav2vec2 model to ONNX model



This notebook show how to convert Thai wav2vec2 model from Huggingface to ONNX model.

Thai wav2vec2 model: [airesearch/wav2vec2-large-xlsr-53-th](https://huggingface.co/airesearch/wav2vec2-large-xlsr-53-th)

## Install

In [ ]:
# !pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.5 MB/s eta 0:10:43tcmalloc: large alloc 1147494400 bytes == 0x55bf21ac6000 @  0x7faf12d1b615 0x55bf1efac4cc 0x55bf1f08c47a 0x55bf1efaf2ed 0x55bf1f0a0e1d 0x55bf1f022e99 0x55bf1f01d9ee 0x55bf1efb0bda 0x55bf1f022d00 0x55bf1f01d9ee 0x55bf1efb0bda 0x55bf1f01f737 0x55bf1f0a1c66 0x55bf1f01edaf 0x55bf1f0a1c66 0x55bf1f01edaf 0x55bf1f0a1c66 0x55bf1f01edaf 0x55bf1efb1039 0x55bf1eff4409 0x55bf1efafc52 0x55bf1f022c25 0x55bf1f01d9ee 0x55bf1efb0bda 0x55bf1f01f737 0x55bf1f01d9ee 0x55bf1efb0bda 0x55bf1f01e915 0x55bf1efb0afa 0x55bf1f01ec0d 0x55bf1f01d9ee
     |██████████████████▌             | 1055.7 MB 1.5 MB/s eta 0:08:37tcmalloc: large alloc 1434370048 bytes == 0x55bf6611c000 @  0x7faf12d1b615 0x55bf1efac4cc 0x55bf1f08c47a 0x55bf1efaf2ed 0x55bf1f0a0e1d 0x55bf1f022e99 0x55bf1f01d9ee 0x55bf1efb0bda 0x55bf1f022d00 0x55bf1f01d9ee 0x55bf1efb0bda 0x55bf1f01f737 0x55bf1f0a1c66 0x55bf

In [ ]:
!pip install transformers onnxruntime onnx pythainlp soundfile

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 4.8 MB 37.3 MB/s 
     |████████████████████████████████| 12.7 MB 91 kB/s 
     |████████████████████████████████| 11.0 MB 2.0 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 3.3 MB 36.0 MB/s 
     |████████████████████████████████| 596 kB 40.1 MB/s 
     |████████████████████████████████| 743 kB 48.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Build ONNX Model

We will build ONNX model.

Resource

- [Build Wav2Vec2Model from Hugging Face to PyTorch](https://pytorch.org/audio/stable/models.html#wav2vec2-0-hubert)
- [Convert your PyTorch model to ONNX](https://docs.microsoft.com/en-us/windows/ai/windows-ml/tutorials/pytorch-convert-model)

In [ ]:
import transformers
from transformers import AutoTokenizer, Wav2Vec2ForCTC
from torchaudio.models.wav2vec2.utils import import_huggingface_model

In [ ]:
original = Wav2Vec2ForCTC.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
imported = import_huggingface_model(original) # Build Wav2Vec2Model from the corresponding model object of Hugging Face https://pytorch.org/audio/stable/models.html#wav2vec2-0-hubert

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:341: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [ ]:
imported.eval() # set the model to inference mode

Wav2Vec2Model(
  (feature_extractor): FeatureExtractor(
    (conv_layers): ModuleList(
      (0): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
      )
      (1): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (2): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (3): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (4): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (5): ConvLayerBlock(
        (lay

In [ ]:
import torch.onnx # https://docs.microsoft.com/en-us/windows/ai/windows-ml/tutorials/pytorch-convert-model

In [ ]:
input_size = 100000
AUDIO_MAXLEN = input_size

In [ ]:
dummy_input = torch.randn(1, input_size, requires_grad=True)

In [ ]:
torch.onnx.export(imported,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         "asr3.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : {0 : 'batch_size'}}) 

/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_helper.py:325: UserWarning: Type cannot be inferred, which might cause exported graph to produce incorrect results.
  warnings.warn("Type cannot be inferred, which might cause exported graph to produce incorrect results.")


## Inference

This onnx inference with onnxruntime.

onnxruntime: [https://onnxruntime.ai/](https://onnxruntime.ai/)

**Load Audio file**

In [ ]:
!wget https://www.dropbox.com/s/9kpeh8eodshcqhj/common_voice_th_23646850.wav?dl=1

In [ ]:
!mv common_voice_th_23646850.wav?dl=1 sound.wav

**load vocab.json from huggingface model**

In [ ]:
!wget https://huggingface.co/airesearch/wav2vec2-large-xlsr-53-th/raw/main/vocab.json

In [ ]:
with open("vocab.json","r",encoding="utf-8-sig") as f:
  d = eval(f.read())

**Inference**

In [ ]:
import onnx
import torch
import onnxruntime

In [ ]:
import numpy as np

In [ ]:
import soundfile as sf
import tensorflow as tf
from scipy.io import wavfile
import scipy.signal as sps
import os
from pythainlp.util import normalize

In [ ]:
input_size = 100000
new_rate = 16000
AUDIO_MAXLEN = input_size

In [ ]:
ort_session = onnxruntime.InferenceSession('asr3.onnx') # load onnx model

In [ ]:
res = dict((v,k) for k,v in d.items())
res[69]="[PAD]"
res[68]="[UNK]"

In [ ]:
def _normalize(x): # 
  """You must call this before padding.
  Code from https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/wav2vec2/processor.py#L101
  Fork TF to numpy
  """
  # -> (1, seqlen)
  mean = np.mean(x, axis=-1, keepdims=True)
  var = np.var(x, axis=-1, keepdims=True)
  return np.squeeze((x - mean) / np.sqrt(var + 1e-5))

In [ ]:
def remove_adjacent(item): # code from https://stackoverflow.com/a/3460423
  nums = list(item)
  a = nums[:1]
  for item in nums[1:]:
    if item != a[-1]:
      a.append(item)
  return ''.join(a)

In [ ]:
def asr(path):
    """
    Code from https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/notebooks/wav2vec2_onnx.ipynb
    Fork TF to numpy
    """
    sampling_rate, data = wavfile.read(path)
    samples = round(len(data) * float(new_rate) / sampling_rate)
    new_data = sps.resample(data, samples)
    speech = np.array(new_data, dtype=np.float32)
    speech = _normalize(speech)[None]
    padding = np.zeros((speech.shape[0], AUDIO_MAXLEN - speech.shape[1]))
    speech = np.concatenate([speech, padding], axis=-1).astype(np.float32)
    ort_inputs = {"modelInput": speech}
    ort_outs = ort_session.run(None, ort_inputs)
    prediction = np.argmax(ort_outs, axis=-1)
    # Text post processing
    _t1 = ''.join([res[i] for i in list(prediction[0][0])])
    return normalize(''.join([remove_adjacent(j) for j in _t1.split("[PAD]")]))

In [ ]:
FILENAME = "sound.wav"

In [ ]:
asr(FILENAME)